## Търсене и извличане на информация. Приложение на дълбоко машинно обучение
> ### Стоян Михов  
> #### Зимен семестър 2021/2022

### Упражнение 2

За да работи програмата трябва да се свали корпус от публицистични текстове за Югоизточна Европа,
предоставен за некомерсиално ползване от Института за български език - БАН
Корпусът може да бъде свален от:
http://dcl.bas.bg/BulNC-registration/dl.php?dl=feeds/JOURNALISM.BG.zip
Архивът трябва да се разархивира в директорията, в която е програмата.

Преди да се стартира програмата на питон е необходимо да се активира съответното обкръжение с командата:
`conda activate tii`

##### Цел: Търсене на документи по думи

In [8]:
!ls JOURNALISM.BG/

C-MassMedia		       JOURNALISM_METADATA.BG.txt
JOURNALISM_DESCRIPTION.BG.pdf  JOURNALISM_METADATA.EN.txt
JOURNALISM_DESCRIPTION.EN.pdf


In [6]:
import nltk
from nltk.corpus import PlaintextCorpusReader
import sys

In [51]:
corpus_root = 'JOURNALISM.BG/C-MassMedia'
myCorpus = PlaintextCorpusReader(corpus_root, '.*\.txt')
fileNames = myCorpus.fileids()

In [52]:
myCorpus.words(fileNames[28])

['Гръцки', 'град', 'бе', 'избран', 'за', 'Европейска', ...]

In [53]:
len(fileNames)

35337

In [16]:
def printDocuments(myCorpus,fileNames,docIDs):
    for docID in docIDs:
        text = myCorpus.words(fileNames[docID])
        print('Document ID: '+str(docID))
        for word in text:
            print(word,end=' ')
        print('\n')

In [29]:
printDocuments(myCorpus,fileNames,[7,28])

Document ID: 7
Министърът на външните работи на Румъния : Демократизацията изисква социално равенство 16 / 11 / 2003 БУКУРЕЩ , Румъния -- Външният министър Мирча Джеоана заяви в събота ( 15 ноември ), че демократизацията изисква да бъде отделяно повече внимание на въпросите на социалното равенство и достъпа на гражданите до обществените услуги . В словото си на регионалната конференция на общността на демокрациите в Букурещ Джеоана каза , че е трудно да се убедят гражданите в достойнствата на демокрацията , ако те изпитват социално неравенство . Той също така посочи , че липсата на прозрачност и корупцията са най - големите заплахи за процеса на демократизация . На конференцията присъстваха представители на ОССЕ , Съвета на Европа и Европейската комисия . ( Ромпрес - 15 / 11 / 03 ) 

Document ID: 28
Земетресението показва нарастващата роля на социалните медии 24 / 05 / 2012 Едно от най - силните земетресения в съвременната история на България предизвика също толкова силен трус в социал

In [20]:
!du -shc *

206M	JOURNALISM.BG
44M	JOURNALISM.BG.zip
204K	temp_upr2.ipynb
8.0K	u2.py
24K	upr2.ipynb
32K	upr2_cl.ipynb
250M	total


###### Как да структурираме информацията?

In [21]:
class progressBar:
    def __init__(self ,barWidth = 80):
        self.barWidth = barWidth
        self.period = None

    def start(self, count):
        self.period = max(int(count / self.barWidth),1)
        sys.stdout.write("["+(" " * self.barWidth)+"]")
        sys.stdout.flush()
        sys.stdout.write("\b" * (self.barWidth+1))

    def tick(self, item):
        if item % self.period == 0:
            sys.stdout.write("-")
            sys.stdout.flush()

    def stop(self):
        sys.stdout.write("]\n")

In [23]:
list(enumerate(['a','b','c']))

[(0, 'a'), (1, 'b'), (2, 'c')]

In [54]:
pb = progressBar()
pb.start(len(fileNames))
dictionary={}
for docID, fileName in enumerate(fileNames):
    pb.tick(docID)
    text = myCorpus.words(fileName)
    for token in text:
        if not token.isalpha(): continue
        term = token.lower()
        if term in dictionary:
            postings = dictionary[term]
            if postings[-1] < docID:
                postings.append(docID)
        else:
            dictionary[term] = [docID]
pb.stop()

[                                                                                ---------------------------------------------------------------------------------]


In [57]:
len(dictionary)

122739

In [59]:
p1 = dictionary['всички'];p1

[4,
 6,
 8,
 9,
 25,
 43,
 46,
 69,
 74,
 75,
 78,
 82,
 124,
 153,
 169,
 171,
 178,
 184,
 187,
 188,
 191,
 194,
 198,
 206,
 212,
 225,
 227,
 268,
 277,
 278,
 279,
 284,
 289,
 291,
 308,
 321,
 331,
 337,
 338,
 339,
 341,
 346,
 368,
 386,
 407,
 441,
 442,
 444,
 447,
 449,
 450,
 451,
 452,
 454,
 466,
 468,
 469,
 471,
 475,
 477,
 481,
 482,
 483,
 486,
 504,
 506,
 515,
 518,
 523,
 544,
 550,
 572,
 580,
 586,
 593,
 595,
 601,
 604,
 609,
 610,
 625,
 636,
 645,
 659,
 663,
 670,
 672,
 678,
 683,
 688,
 689,
 692,
 696,
 705,
 708,
 739,
 759,
 761,
 765,
 767,
 768,
 777,
 781,
 794,
 823,
 825,
 831,
 832,
 838,
 840,
 850,
 852,
 862,
 865,
 876,
 885,
 888,
 890,
 898,
 901,
 907,
 911,
 918,
 920,
 951,
 963,
 970,
 973,
 993,
 994,
 1008,
 1009,
 1020,
 1022,
 1053,
 1054,
 1059,
 1071,
 1075,
 1079,
 1085,
 1088,
 1090,
 1092,
 1093,
 1097,
 1099,
 1100,
 1102,
 1103,
 1104,
 1105,
 1112,
 1117,
 1122,
 1124,
 1126,
 1127,
 1132,
 1133,
 1134,
 1136,
 1142,
 1163

In [37]:
p2 = dictionary['хора']

##### Да реализираме операциите за сливане:

In [35]:
def intersect(p1,p2):
    i1=0
    i2=0
    answer=[]
    while i1<len(p1) and i2<len(p2):
        if p1[i1] == p2[i2]:
            answer.append(p1[i1])
            i1 += 1
            i2 += 1
        elif p1[i1] < p2[i2]:
            i1 += 1
        else:
            i2 += 1
    return answer

In [39]:
p1

[11, 13, 14, 16, 20, 22, 26, 27, 28]

In [40]:
p2

[0, 4, 11, 12, 13, 23, 24, 26, 27, 28]

In [38]:
intersect(p1,p2)

[11, 13, 26, 27, 28]

##### Сечение на повече от един списък?

In [41]:
def intersectLists(andLists):
    if not andLists: return []
    postingLists = sorted(andLists, key = lambda x : len(x))
    answer = postingLists[0]
    for i in range(1,len(postingLists)):
        answer = intersect(answer,postingLists[i])
    return answer

In [42]:
def andSearch(dictionary, termList):
    dicItems = [ dictionary[term] for term in termList if term in dictionary ]
    return intersectLists(dicItems)

In [49]:
andSearch(dictionary,['бизнес'])

[0, 19, 20]

In [50]:
def union(p1,p2):
    i1 = 0
    i2 = 0
    answer = []
    while i1 < len(p1) or i2 < len(p2):
        if i2 == len(p2) or (i1 < len(p1) and p1[i1] < p2[i2]):
            answer.append(p1[i1])
            i1 += 1
        elif i1 == len(p1) or (i2 < len(p2) and p2[i2] < p1[i1]):
            answer.append(p2[i2])
            i2 += 1
        else:
            answer.append(p1[i1])
            i1 += 1
            i2 += 1
    return answer

##### Цел: търсене на документи по шаблони

In [60]:
class Trie:
    def __init__(self):
        self.stateTransitions = [{}]
        self.finalStates = set([])

    def traverse(self,word,initialState = 0):
        s = initialState # current state
        i = 0 # current position in word
        while i < len(word) and word[i] in self.stateTransitions[s]:
            s = self.stateTransitions[s][word[i]]
            i += 1
        return s, i

    def isFinal(self, s):
        return s in self.finalStates

    def inTrie(self, word):
        s, i = self.traverse(word)
        return i == len(word) and self.isFinal(s)

    def addWord(self,word):
        s, i = self.traverse(word) # s = 8; i->ч
        while i < len(word):
            newState = len(self.stateTransitions) # 14
            self.stateTransitions.append({})
            st = self.stateTransitions[s]
            st[word[i]] = newState
            s = newState
            i += 1
        self.finalStates.add(s)

    def getWordsFromState(self,s):
        answer = []
        if s in self.finalStates:
            answer.append('')
        st = self.stateTransitions[s]
        for (char,nextstate) in st.items():
            answer += [ char + word for word in self.getWordsFromState(nextstate) ]
        return answer

    def getWordsWithPrefix(self,prefix):
        s,i = self.traverse(prefix)
        if i != len(prefix):
            return []
        else:
            return [ prefix + word for word in self.getWordsFromState(s) ]

In [64]:
dictionaryTrie=Trie()

In [65]:
len(dictionary)

122739

In [73]:
len(dictionaryTrie.stateTransitions)

293480

In [75]:
getsize(dictionary)/1024/1024

62.24669075012207

In [77]:
getsize(dictionaryTrie)/1024/1024

86.56922817230225

In [68]:
for term in list(dictionary):
    dictionaryTrie.addWord(term)

In [78]:
def andSearchWithWildCards(dictionary,dictionaryTrie,patternList):
    items = []
    for pattern in patternList:
        orList = []
        for term in dictionaryTrie.getWordsWithPrefix(pattern):
            orList = union(orList,dictionary[term])
        items.append(orList)
    return intersectLists(items)

In [79]:
andSearchWithWildCards(dictionary, dictionaryTrie, ['култур','обмен','теат'])

[128, 138, 139, 140, 176, 179, 246, 265, 276, 337, 442, 19885, 31590, 32403]

In [80]:
printDocuments(myCorpus,fileNames,[128, 138, 139, 140, 176, 179, 246, 265, 276, 337, 442, 19885, 31590, 32403])

Document ID: 128
Културен преглед 23 / 02 / 2005 Преглед на спортните , развлекателните и други културни новини от Балканите след 16 февруари ( Различни източници – 16 / 02 / 05 – 23 / 02 / 05 ) Албанският национален театър за опера и балет гастролира за пръв път в Македония на 18 февруари с операта " Любовно биле " от Гаетано Доницети в Скопие . Базираната в Македония фондация " Балкани без граници " финансира тази проява и ще спонсорира предстоящ концерт в Тирана на Македонския оперен театър . [ Toмислав Георгиев ] Три изложби се представят понастоящем в Националната галерия по изкуствата в Тирана . Първата , " Синапси ", предлага творби на млади албански и италиански художници . Отделна изложба представя младия албански творец Елира Суло , а фотоизложбата с произведения на германския фотограф Регина Шмекен ще бъде отворена за публиката до края на февруари . *** Хърватия и Йордания подписаха споразумение за сътрудничество в областта на културата по време на визитата на премиера Иво С

Културен обзор 28 / 12 / 2005 Преглед на спортните , забавни и културни новини в Югоизточна Европа след 21 декември . ( Различни източници – 21 / 12 / 05 – 28 / 12 / 05 ) Ансамбълът за фолклорни танци и песни на Македония " Танец " по време на представление на Новогодишния концерт в Македонската опера и балет в Скопие в събота ( 25 декември ). [ AФП ] Националният театър на Сараево и Националният театър на Сърбия в Нови Сад са постигнали споразумение за сътрудничество и обмен на две представления през първата половина на 2006 г ., съобщи директорът на театъра на Сараево Градимир Гойер миналата седмица . Той добави , че също така се планира турне на сараевския балет и опера в Нови сад като част от програмата за сътрудничество . *** Нападателят на Байер Леверкузен , Димитър Бербатов бе определен за футболист на годината на България , спечелвайки годишната анкета проведена сред спортни журналисти в страната . Втори бе хълфът на Селтик Стилиян Петров . Церемонията по награждаването се пров

Култура и спорт : Проведе се 25 - тият Атински класически маратон 07 / 11 / 2007 Атинският класически маратон се проведе в Гърция . Още новини от седмицата : Кинофестивалът " Киномания " започна в София и се провежда конкурс за най - красивата дума в света . Участник в Атинския класически маратон 2007 в неделя ( 4 ноември ) [ Гети Имиджис ] Бенджамин Кипротич Корин от Кения и рускинята Светлана Пономаренко спечелиха 25 - тия Атински класически маратон в гръцката столица в неделя ( 4 ноември . Маратонът следва маршрута на състезанието от древността , като започва от крайбрежния град Маратон и свършва на стадион " Панатинайкон " в Атина . *** Tурската дума yakamoz , която означава " отражение на луната във водата ", спечели състезанието за най - красива дума в света . Събитието бе организирано от германското електронно списание KulturAustausch ( Културен обмен ). *** Семинар по дизайн , организиран от швейцарската културна програма в Maкедония , започна в понеделник ( 5 ноември ) в Скопи

Македонски детски театър носи радост 04 / 08 / 2010 Културата е сила , която може да преодолява предразсъдъците и да прекосява границите , казва режисьорът Любомир Чадиковски . От Марина Стояновска за Southeast European Times от Скопие -- 04 / 08 / 10 Режисьорът Любомир Чадиковски работи в театъра от неговото създаване преди повече от 20 години . [ Томислав Георгиев / SETimes ] В продължение на повече от 20 години младите хора масово посещават наградените постановки на македонския Театър за деца и юноши , навлизайки с света на фантазиите , емоциите и фолклора . С репертоар , включващ пиеси като " Красавицата и звяра ", " Малкият принц ", " Малката Русалка " и " Крали Марко " театърът продължава да омайва своята публика . Кореспондентът на SETimes Марина Стояновска беседва с режисьора Любомир Чадиковски , който работи в театъра от неговото създаване . SETimes : Какво е чувството да чествате 20 - годишнината на театъра ? И как поддържате интереса сред най - трудната публика , децата ? Лю

##### getsize function:

In [56]:
import sys
from numbers import Number
from collections import deque
from collections.abc import Set, Mapping


ZERO_DEPTH_BASES = (str, bytes, Number, range, bytearray)


def getsize(obj_0):
    """Recursively iterate to sum size of object & members."""
    _seen_ids = set()
    def inner(obj):
        obj_id = id(obj)
        if obj_id in _seen_ids:
            return 0
        _seen_ids.add(obj_id)
        size = sys.getsizeof(obj)
        if isinstance(obj, ZERO_DEPTH_BASES):
            pass # bypass remaining control flow and return
        elif isinstance(obj, (tuple, list, Set, deque)):
            size += sum(inner(i) for i in obj)
        elif isinstance(obj, Mapping) or hasattr(obj, 'items'):
            size += sum(inner(k) + inner(v) for k, v in getattr(obj, 'items')())
        # Check for custom object instances - may subclass above too
        if hasattr(obj, '__dict__'):
            size += inner(vars(obj))
        if hasattr(obj, '__slots__'): # can have __slots__ with __dict__
            size += sum(inner(getattr(obj, s)) for s in obj.__slots__ if hasattr(obj, s))
        return size
    return inner(obj_0)